In [1]:
# --- 匯入套件 ---
import os
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

In [2]:
# =========================================================
# Step 1: 批次讀取多個 CSV 檔案
# =========================================================
data_dir = "./dance_csv/"  # ← 這裡改成放你CSV檔的資料夾
all_poses = []

def parse_point(s):
    s = s.strip("()")
    parts = [float(p.strip(" '")) for p in s.split(",")]
    return parts

for file in os.listdir(data_dir):
    if file.endswith(".csv"):
        csv_path = os.path.join(data_dir, file)
        df = pd.read_csv(csv_path)
        pose_cols = [c for c in df.columns if c != "frame"]
        poses = []
        for _, row in df.iterrows():
            pose = []
            for c in pose_cols:
                pose += parse_point(row[c])
            poses.append(pose)
        all_poses.append(np.array(poses))
        print(f"✅ 已載入：{file}, 共 {len(poses)} 幀")

✅ 已載入：Ballet_1.csv, 共 241 幀
✅ 已載入：Ballet_10.csv, 共 70 幀
✅ 已載入：Ballet_11.csv, 共 832 幀
✅ 已載入：Ballet_12.csv, 共 9 幀
✅ 已載入：Ballet_13.csv, 共 18 幀
✅ 已載入：Ballet_14.csv, 共 1167 幀
✅ 已載入：Ballet_15.csv, 共 3 幀
✅ 已載入：Ballet_16.csv, 共 48 幀
✅ 已載入：Ballet_17.csv, 共 481 幀
✅ 已載入：Ballet_18.csv, 共 61 幀
✅ 已載入：Ballet_19.csv, 共 229 幀
✅ 已載入：Ballet_2.csv, 共 15 幀
✅ 已載入：Ballet_20.csv, 共 55 幀
✅ 已載入：Ballet_21.csv, 共 224 幀
✅ 已載入：Ballet_22.csv, 共 369 幀
✅ 已載入：Ballet_23.csv, 共 79 幀
✅ 已載入：Ballet_24.csv, 共 319 幀
✅ 已載入：Ballet_25.csv, 共 155 幀
✅ 已載入：Ballet_26.csv, 共 171 幀
✅ 已載入：Ballet_27.csv, 共 25 幀
✅ 已載入：Ballet_28.csv, 共 79 幀
✅ 已載入：Ballet_29.csv, 共 1283 幀
✅ 已載入：Ballet_3.csv, 共 1832 幀
✅ 已載入：Ballet_30.csv, 共 583 幀
✅ 已載入：Ballet_31.csv, 共 389 幀
✅ 已載入：Ballet_32.csv, 共 554 幀
✅ 已載入：Ballet_33.csv, 共 182 幀
✅ 已載入：Ballet_34.csv, 共 86 幀
✅ 已載入：Ballet_35.csv, 共 78 幀
✅ 已載入：Ballet_36.csv, 共 447 幀
✅ 已載入：Ballet_37.csv, 共 347 幀
✅ 已載入：Ballet_38.csv, 共 76 幀
✅ 已載入：Ballet_39.csv, 共 167 幀
✅ 已載入：Ballet_4.csv, 共 58 幀
✅ 已載入：Ballet_40.csv, 共 593 幀
✅ 已

In [3]:
# 合併所有 CSV
all_poses = np.concatenate(all_poses, axis=0)
print("📦 全部骨架資料 shape:", all_poses.shape)  # (總幀數, 99)

📦 全部骨架資料 shape: (271194, 99)


In [4]:
# =========================================================
# Step 2: 正規化
# =========================================================
scaler = StandardScaler()
all_poses = scaler.fit_transform(all_poses)

In [5]:
# =========================================================
# Step 3: 定義 Dataset / DataLoader
# =========================================================
class PoseDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.float32)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

dataset = PoseDataset(all_poses)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

In [6]:
# =========================================================
# Step 4: 定義模型架構（需與訓練時一致）
# =========================================================
class VectorQuantizer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.num_embeddings = num_embeddings
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.uniform_(-1/num_embeddings, 1/num_embeddings)

    def forward(self, x):
        distances = (
            torch.sum(x**2, dim=1, keepdim=True)
            + torch.sum(self.embedding.weight**2, dim=1)
            - 2 * torch.matmul(x, self.embedding.weight.t())
        )
        encoding_indices = torch.argmin(distances, dim=1)
        quantized = self.embedding(encoding_indices)
        return quantized, encoding_indices

In [7]:
class VQVAE(nn.Module):
    def __init__(self, input_dim=99, hidden_dim=128, latent_dim=32, num_embeddings=64):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )
        self.vq = VectorQuantizer(num_embeddings, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        z_q, indices = self.vq(z)
        x_recon = self.decoder(z_q)
        return x_recon, indices, z, z_q

In [8]:
# =========================================================
# Step 5: 載入訓練好的模型
# =========================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
model = VQVAE().to(device)

model_path = "vqvae_model.pth"  # ← 你的模型權重檔案
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location=device))
    print(f"✅ 已載入模型權重：{model_path}")
else:
    raise FileNotFoundError(f"❌ 找不到模型檔案：{model_path}")

model.eval()

✅ 已載入模型權重：vqvae_model.pth


C:\Users\user\AppData\Local\Temp\ipykernel_27440\2112891260.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=dev

VQVAE(
  (encoder): Sequential(
    (0): Linear(in_features=99, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=32, bias=True)
  )
  (vq): VectorQuantizer(
    (embedding): Embedding(64, 32)
  )
  (decoder): Sequential(
    (0): Linear(in_features=32, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=99, bias=True)
  )
)

In [9]:
# =========================================================
# Step 6: 生成符號序列
# =========================================================
all_indices = []

with torch.no_grad():
    for batch in dataloader:
        batch = batch.to(device)
        _, indices, _, _ = model(batch)
        all_indices.extend(indices.cpu().numpy().tolist())

In [10]:
# 把每個 index 映射成符號 (A-Z, a-z, 0-9 循環)
def index_to_symbol(i):
    symbols = [chr(c) for c in range(65, 91)] + [chr(c) for c in range(97, 123)] + [str(d) for d in range(10)]
    return symbols[i % len(symbols)]

symbol_sequence = [index_to_symbol(i) for i in all_indices]

In [11]:
# =========================================================
# Step 7: 儲存成 JSON
# =========================================================
output_path = "symbol_sequences.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(symbol_sequence, f, ensure_ascii=False, indent=2)

print(f"✅ 已輸出符號序列 JSON：{output_path}")
print("符號預覽：", "".join(symbol_sequence[:200]))

✅ 已輸出符號序列 JSON：symbol_sequences.json
符號預覽： eeeeeeeeeeiieeiieeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeieeeeeeeeeeeeeeeeeeeeeeeeeeeeeeiiiiiiiiiiiiiiiieeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeiiiiii
